# Segmenting and Clustering Neighborhoods in Toronto

## Task 1

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from bs4 import BeautifulSoup
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
# !conda install -c conda-forge scikit-learn
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
can_postals = requests.get(url)
can_postals

<Response [200]>

In [4]:
soup = BeautifulSoup(can_postals.text, 'html.parser')

In [9]:
# can_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

# for row in soup.find("tbody").find_all("tr"):
# #     col = row.find_all("td")
# #     PostalCode = col[0].text.rstrip("\n")
# #     Borough = col[1].text.rstrip("\n")
# #     Neighborhood = col[2].text.rstrip("\n")
# #     col = row.find("td")
# #     PostalCode = col.next_sibling
# #     Borough = PostalCode.next_sibling
# #     Neighborhood = Borough.next_sibling
#     col = row.find("td")
#     PostalCode = col.next_sibling.replace("\n", "")
#     Borough = col.next_sibling.next_sibling.text
#     Neighborhood = col.next_sibling.next_sibling.next_sibling
    
#     can_data = can_data.append({"PostalCode":PostalCode, "Borough":Borough, "Neighborhood":Neighborhood}, ignore_index=True)
# can_data.head(10)

In [6]:
# open Wiki page with Beautiful Soup
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [13]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(100)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [11]:
# To check whether we have empty, NA or NaN values
df.Borough.value_counts()

North York                24
Scarborough               17
Downtown Toronto          17
Etobicoke                 11
Central Toronto            9
West Toronto               6
York                       5
East Toronto               4
East York                  4
Downtown Toronto Stn A     1
Etobicoke Northwest        1
East Toronto Business      1
Mississauga                1
Queen's Park               1
East York/East Toronto     1
Name: Borough, dtype: int64

In [17]:
df.Neighborhood.isnull().sum(), df.Neighborhood.str.count("Not assigned").sum()

(0, 0)

So far, so good!

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PostalCode    103 non-null    object
 1   Borough       103 non-null    object
 2   Neighborhood  103 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [15]:
df.shape

(103, 3)

## Task 2


In [19]:
    !pip install geocoder

  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=18a3d6b4dfee1215ee28acbdb1dc0ae2df1c0f47fdb3bb349423db36eee919d0
  Stored in directory: c:\users\crusa\appdata\local\pip\cache\wheels\8e\70\28\3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future


In [24]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coo = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [25]:
coo.shape

(103, 3)

In [26]:
coo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Postal Code  103 non-null    object 
 1   Latitude     103 non-null    float64
 2   Longitude    103 non-null    float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


Before merging (joining) the two tables we need to rename the first column from `Postal Code` to `PostalCode`, so that the merging will be possible.

In [27]:
coo = coo.rename(columns={'Postal Code': 'PostalCode'})

In [30]:
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [32]:
df = pd.merge(df, coo, on=["PostalCode"])
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Task 3

Just like what we did in the neighbourhood of NYC, we use clustering on Toronto using KMeans.

In [33]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


In [42]:
# Creating the map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for latitude, longitude, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=7,
        popup=label,
        color='green',
        fill=True,
        fill_color="yellow",
        fill_opacity = 0.2
        ).add_to(map_Toronto)  
    
map_Toronto

Now just like New York Neighborhood's notebook we set our Foursquare credentials intact.

In [43]:
CLIENT_ID = '1W4ISRYVBTRFFKK2TEAOMD0J5H2EO5ED00WXY4KS2N4PLX5Z' # your Foursquare ID
CLIENT_SECRET = 'TON54IBIBGQEXUGLR5ZD5S0Z2DF1WGCR5HSBGZVEAJOSWDAU' # your Foursquare Secret
ACCESS_TOKEN = 'E3QMI1FJRJUHLUUZCSLDYRMVOR4KMWYF3VEEZHC420HKUDFG' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LDIJF4KI5VGMMA3NNDLFZWHR12TCMNTUL0TUC3QPZ3SJD040
CLIENT_SECRET:EOOOZ3EF5N0FOMNUJVTDV0SXVUVVEBMWPFXMNBK1R5K4H55A


Next, we create a function to get all the venue categories in Toronto


In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
requests.get(url).json

<bound method Response.json of <Response [200]>>

Collecting the venues in Toronto for each Neighbourhood



In [50]:
venues_in_toronto = getNearbyVenues(df['Neighborhood'], df['Latitude'], df['Longitude'])

Parkwoods


KeyError: 'groups'

In [48]:
venues_in_toronto.shape

NameError: name 'venues_in_toronto' is not defined